In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/narou
!ls -a
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/narou
all_metadata.csv	    dimension_reducer_v1.pth  poolformer_v0.ipynb
array_reduce.ipynb	    dimension_reducer_v2.pth  poolformer_v1.ipynb
bin_corpus_v1		    encoder_v1.pth	      poolformer_v2.ipynb
corpus-50000-71750.tar.bz2  encoder_v2.pth	      pth_list.txt
corpus_v1		    log.txt		      terminal.ipynb
corpus_v2		    ncodes_v1.txt	      text_binary.ipynb
data_v0.5.csv		    ncode_v2.txt	      VAE.ipynb
/content/drive/MyDrive/narou


In [ ]:
import torch
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime
Sun Aug 13 05:23:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    31W /  70W |   1085MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
         

In [ ]:
!pip install tqdm
import torch
import numpy as np
import torch.nn as nn
import os
from tqdm import tqdm

In [ ]:
class Encoder(nn.Module):
    def __init__(self, z_dim=32, in_dim=768):
        super().__init__()
        self.lr = nn.Linear(in_dim, int(in_dim//2))
        self.lr_ave = nn.Linear(int(in_dim//2), z_dim)   #average
        self.lr_dev = nn.Linear(int(in_dim//2), z_dim)   #log(sigma^2)
        self.relu = nn.ReLU()
        self.layer_norm = nn.LayerNorm(in_dim)

    def forward(self, x):
        x = self.layer_norm(x)
        x = self.lr(x)
        x = self.relu(x)
        ave = self.lr_ave(x)    #average
        log_dev = self.lr_dev(x)    #log(sigma^2)

        ep = torch.randn_like(ave)   #平均0分散1の正規分布に従い生成されるz_dim次元の乱数
        z = ave + torch.exp(log_dev / 2) * ep   #再パラメータ化トリック
        return z

class Decoder(nn.Module):
    def __init__(self, z_dim=32, in_dim=768):
        super().__init__()
        self.lr = nn.Linear(z_dim, in_dim//2)
        self.lr2 = nn.Linear(in_dim//2, in_dim)
        self.relu = nn.ReLU()
        self.layer_norm = nn.LayerNorm(z_dim)

    def forward(self, x):
        x = self.layer_norm(x)
        x = self.lr(x)
        x = self.relu(x)
        x = self.lr2(x)


        return x

class VAE(nn.Module):
    def __init__(self, z_dim=32, in_dim=768 ):
        super().__init__()
        self.encoder = Encoder(z_dim=z_dim, in_dim=in_dim)
        self.decoder = Decoder(z_dim=z_dim, in_dim=in_dim)

    def forward(self, x):
            z = self.encoder(x)
            x = self.decoder(z)
            return x, z


train

In [11]:
def main(device):

    model = torch.load("dimension_reducer_v2.pth").to(device)
    corpus = "/content/drive/MyDrive/narou/bin_corpus_v1/"
    txts = os.listdir(corpus)

    model.train()
    total_loss_train=0.0
    n=0
    for txt in tqdm(txts):

        input=np.load(corpus+txt)
        if input.shape[1] == 768:
            pass
        else:
            continue
        torch.cuda.empty_cache()
        input=torch.tensor(input[:512*144])
        input=torch.unsqueeze(input, 0).to(device)
        output, hid=model(input)
        #print(f'input:{input}, hid:{hid.shape}, output:{output}')
        del output
        hid=torch.squeeze(hid, 0)
        hid=hid.to('cpu').detach().numpy().copy()
        np.save(corpus+txt, hid)


if __name__ == "__main__":
    main(device)


100%|██████████| 4234/4234 [2:07:30<00:00,  1.81s/it]
